# `pip install` &  `wget`

In [149]:
!pip install duckdb==1.0.0
!pip list | grep duckdb
!pip install polars==1.3.0
!pip list | grep polars

duckdb                           1.0.0
polars                           1.3.0


In [150]:
!curl -IL https://3253-machine-learning.s3.ca-central-1.amazonaws.com/lit_panel_aggr_v3.duckdb
!time wget --no-clobber https://3253-machine-learning.s3.ca-central-1.amazonaws.com/lit_panel_aggr_v3.duckdb
!time wget --no-clobber https://3253-machine-learning.s3.ca-central-1.amazonaws.com/market/aapl_daily_historical-data-07-27-2024.csv
!ls -lah

HTTP/1.1 200 OK
x-amz-id-2: nDHSfqQF0Nph/Lx8HTLxNnUmYzd4yceEe9SBtkPyS8fqvJ7grhL7eEdNKt7Dd6xELFtu3VpRga4=
x-amz-request-id: 648PRMAGHRPN0WXE
Date: Mon, 29 Jul 2024 04:13:11 GMT
Last-Modified: Fri, 26 Jul 2024 03:24:09 GMT
ETag: "3f08ffe0a0433c624fb7b08a95ab7ff8-87"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: binary/octet-stream
Server: AmazonS3
Content-Length: 728248320

File ‘lit_panel_aggr_v3.duckdb’ already there; not retrieving.


real	0m0.006s
user	0m0.002s
sys	0m0.002s
File ‘aapl_daily_historical-data-07-27-2024.csv’ already there; not retrieving.


real	0m0.005s
user	0m0.000s
sys	0m0.004s
total 695M
drwxr-xr-x 1 root root 4.0K Jul 29 02:23 .
drwxr-xr-x 1 root root 4.0K Jul 29 02:15 ..
-rw-r--r-- 1 root root 382K Jul 27 21:07 aapl_daily_historical-data-07-27-2024.csv
drwxr-xr-x 4 root root 4.0K Jul 25 13:20 .config
-rw-r--r-- 1 root root 695M Jul 26 03:24 lit_panel_aggr_v3.duckdb
drwxr-xr-x 1 root root 4.0K Jul 25 13:21 sample_data


# DuckDB Connection and Schema Exploration

In [151]:
import duckdb

conn = duckdb.connect('lit_panel_aggr_v3.duckdb')
conn.sql("SELECT * FROM lit_panel_aggr_v3 LIMIT 1").df().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 27 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   URL                                  1 non-null      object 
 1   acceptanceDatetime                   1 non-null      int64  
 2   accessionNumber                      1 non-null      object 
 3   type                                 1 non-null      object 
 4   period                               1 non-null      int64  
 5   issuerCIK                            1 non-null      object 
 6   documentType                         1 non-null      object 
 7   periodOfReport                       1 non-null      object 
 8   notSubjectToSection16                0 non-null      object 
 9   issuerTradingSymbol                  1 non-null      object 
 10  m_isDirector                         1 non-null      int64  
 11  m_isOfficer                         

# Load AAPL panel data from DuckDB

In [152]:
aapl_panel = conn.sql("""SELECT period, transactionCode, _total_transaction_shares, _total_transaction_amount,
                          -- _minSharesOwnedFollowingTransaction as _minSOFT, _maxSharesOwnedFollowingTransaction as _maxSOFT,
                          _count
                        FROM lit_panel_aggr_v3
                        WHERE issuerTradingSymbol = 'AAPL' AND transactionType = 'nonDerivativeTransaction'
                        ORDER BY period DESC""").df()

aapl_panel.info()
display(aapl_panel)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1735 entries, 0 to 1734
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   period                     1735 non-null   int64  
 1   transactionCode            1735 non-null   object 
 2   _total_transaction_shares  1735 non-null   float64
 3   _total_transaction_amount  1344 non-null   float64
 4   _count                     1735 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 67.9+ KB


,period,transactionCode,_total_transaction_shares,_total_transaction_amount,_count
0,20240530,S,75000.0,1.436850e+07,1
1,20240515,S,4999.0,9.517846e+05,1
2,20240510,G,1850.0,0.000000e+00,1
3,20240415,M,8119.0,NaN,1
4,20240415,F,3120.0,5.387928e+05,1
...,...,...,...,...,...
1730,20040419,M,250000.0,1.689070e+06,3
1731,20040419,S,678400.0,1.904628e+07,3
1732,20040419,M,200000.0,2.939065e+06,2
1733,20040419,S,200000.0,5.600000e+06,2


# Load AAPL market data from CSV

In [153]:
import pandas as pd

aapl_market = pd.read_csv('aapl_daily_historical-data-07-27-2024.csv', skipfooter=1)

aapl_market.info()
display(aapl_market)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6180 entries, 0 to 6179
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    6180 non-null   object 
 1   Open    6180 non-null   float64
 2   High    6180 non-null   float64
 3   Low     6180 non-null   float64
 4   Last    6180 non-null   float64
 5   Change  6180 non-null   float64
 6   %Chg    6180 non-null   object 
 7   Volume  6180 non-null   int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 386.4+ KB


<ipython-input-153-cedac91fdc44>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  aapl_market = pd.read_csv('aapl_daily_historical-data-07-27-2024.csv', skipfooter=1)


,Time,Open,High,Low,Last,Change,%Chg,Volume
0,07/26/2024,218.7000,219.4900,216.0100,217.9600,0.4700,+0.22%,41601300
1,07/25/2024,218.9300,220.8500,214.6200,217.4900,-1.0500,-0.48%,51391200
2,07/24/2024,224.0000,224.8000,217.1300,218.5400,-6.4700,-2.88%,61777500
3,07/23/2024,224.3650,226.9400,222.6800,225.0100,1.0500,+0.47%,39960200
4,07/22/2024,227.0100,227.7800,223.0900,223.9600,-0.3500,-0.16%,48201800
...,...,...,...,...,...,...,...,...
6175,01/07/2000,0.8616,0.9018,0.8527,0.8884,0.0402,+4.74%,460734432
6176,01/06/2000,0.9476,0.9554,0.8482,0.8482,-0.0804,-8.66%,767984128
6177,01/05/2000,0.9263,0.9871,0.9196,0.9286,0.0134,+1.46%,778321728
6178,01/04/2000,0.9665,0.9878,0.9035,0.9152,-0.0843,-8.43%,511728096


# Extend market data with `datetime` index and running %-change columns

-  Use `%Chg:1d` to confirm perfect match with `%Chg` from source CSV up to rounding ✅

- Parse CSV field `Time` as `%m/%d/%Y` into `parsed_datetime` and set as index

In [154]:

aapl_market['%Chg:1d'] = (aapl_market['Last'] - aapl_market['Last'].shift(-1))/aapl_market['Last'].shift(-1) * 100
aapl_market['%Chg:5d'] = (aapl_market['Last'] - aapl_market['Last'].shift(-5))/aapl_market['Last'].shift(-5) * 100
aapl_market['%Chg:30d'] = (aapl_market['Last'] - aapl_market['Last'].shift(-30))/aapl_market['Last'].shift(-30) * 100

aapl_market['parsed_datetime'] = pd.to_datetime(aapl_market['Time'], format='%m/%d/%Y', errors='raise')
aapl_market.set_index('parsed_datetime', inplace=True)
aapl_market.drop(columns=['Open', 'High', 'Low'], inplace=True)

aapl_market.info()
display(aapl_market)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6180 entries, 2024-07-26 to 2000-01-03
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Time      6180 non-null   object 
 1   Last      6180 non-null   float64
 2   Change    6180 non-null   float64
 3   %Chg      6180 non-null   object 
 4   Volume    6180 non-null   int64  
 5   %Chg:1d   6179 non-null   float64
 6   %Chg:5d   6175 non-null   float64
 7   %Chg:30d  6150 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 434.5+ KB


,Time,Last,Change,%Chg,Volume,%Chg:1d,%Chg:5d,%Chg:30d
parsed_datetime,,,,,,,,
2024-07-26,07/26/2024,217.9600,0.4700,+0.22%,41601300,0.216102,-2.830904,2.295020
2024-07-25,07/25/2024,217.4900,-1.0500,-0.48%,51391200,-0.480461,-2.984209,4.991552
2024-07-24,07/24/2024,218.5400,-6.4700,-2.88%,61777500,-2.875428,-4.517651,13.162800
2024-07-23,07/23/2024,225.0100,1.0500,+0.47%,39960200,0.468834,-4.177668,14.282086
2024-07-22,07/22/2024,223.9600,-0.3500,-0.16%,48201800,-0.156034,-4.453925,15.158371
...,...,...,...,...,...,...,...,...
2000-01-07,01/07/2000,0.8884,0.0402,+4.74%,460734432,4.739448,NaN,NaN
2000-01-06,01/06/2000,0.8482,-0.0804,-8.66%,767984128,-8.658195,NaN,NaN
2000-01-05,01/05/2000,0.9286,0.0134,+1.46%,778321728,1.464161,NaN,NaN


# Extend panel data with `datetime` index

- Parse SQL field `period` as `%Y%m%d` into `parsed_datetime` and set as index

In [155]:

# https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
# %d - Day of the month as a zero-padded decimal number: 01, 02, …, 31
# %m - Month as a zero-padded decimal number: 01, 02, …, 12
# %Y - Year with century as a decimal number: 0001, 0002, …, 2013, 2014, …, 9998, 9999
aapl_panel['parsed_datetime'] = pd.to_datetime(aapl_panel['period'], format='%Y%m%d')
aapl_panel.set_index('parsed_datetime', inplace=True)

aapl_panel.info()
display(aapl_panel)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1735 entries, 2024-05-30 to 2003-09-26
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   period                     1735 non-null   int64  
 1   transactionCode            1735 non-null   object 
 2   _total_transaction_shares  1735 non-null   float64
 3   _total_transaction_amount  1344 non-null   float64
 4   _count                     1735 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 81.3+ KB


,period,transactionCode,_total_transaction_shares,_total_transaction_amount,_count
parsed_datetime,,,,,
2024-05-30,20240530,S,75000.0,1.436850e+07,1
2024-05-15,20240515,S,4999.0,9.517846e+05,1
2024-05-10,20240510,G,1850.0,0.000000e+00,1
2024-04-15,20240415,M,8119.0,NaN,1
2024-04-15,20240415,F,3120.0,5.387928e+05,1
...,...,...,...,...,...
2004-04-19,20040419,M,250000.0,1.689070e+06,3
2004-04-19,20040419,S,678400.0,1.904628e+07,3
2004-04-19,20040419,M,200000.0,2.939065e+06,2


# JOIN panel with market data on parsed `datetime` indexes!

In [156]:
aapl_join = aapl_panel.join(aapl_market, on='parsed_datetime')

aapl_join.info()
display(aapl_join)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1735 entries, 2024-05-30 to 2003-09-26
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   period                     1735 non-null   int64  
 1   transactionCode            1735 non-null   object 
 2   _total_transaction_shares  1735 non-null   float64
 3   _total_transaction_amount  1344 non-null   float64
 4   _count                     1735 non-null   int64  
 5   Time                       1432 non-null   object 
 6   Last                       1432 non-null   float64
 7   Change                     1432 non-null   float64
 8   %Chg                       1432 non-null   object 
 9   Volume                     1432 non-null   float64
 10  %Chg:1d                    1432 non-null   float64
 11  %Chg:5d                    1432 non-null   float64
 12  %Chg:30d                   1432 non-null   float64
dtypes: float64(8), int64(2), objec

,period,transactionCode,_total_transaction_shares,_total_transaction_amount,_count,Time,Last,Change,%Chg,Volume,%Chg:1d,%Chg:5d,%Chg:30d
parsed_datetime,,,,,,,,,,,,,
2024-05-30,20240530,S,75000.0,1.436850e+07,1,05/30/2024,191.2900,1.0000,+0.53%,49947900.0,0.525514,0.204295,13.863095
2024-05-15,20240515,S,4999.0,9.517846e+05,1,05/15/2024,189.7200,2.2900,+1.22%,70399900.0,1.221789,3.819634,11.830239
2024-05-10,20240510,G,1850.0,0.000000e+00,1,05/10/2024,183.0500,-1.5200,-0.82%,50759400.0,-0.823536,-0.179954,6.747143
2024-04-15,20240415,M,8119.0,NaN,1,04/15/2024,172.6900,-3.8600,-2.19%,73531700.0,-2.186349,2.517067,-3.879550
2024-04-15,20240415,F,3120.0,5.387928e+05,1,04/15/2024,172.6900,-3.8600,-2.19%,73531700.0,-2.186349,2.517067,-3.879550
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-04-19,20040419,M,250000.0,1.689070e+06,3,04/19/2004,0.5062,-0.0149,-2.86%,712348096.0,-2.859336,1.098462,6.010471
2004-04-19,20040419,S,678400.0,1.904628e+07,3,04/19/2004,0.5062,-0.0149,-2.86%,712348096.0,-2.859336,1.098462,6.010471
2004-04-19,20040419,M,200000.0,2.939065e+06,2,04/19/2004,0.5062,-0.0149,-2.86%,712348096.0,-2.859336,1.098462,6.010471
